# Buurtscore project - maken datamodel 2017 voor analyse

### Beschrijving
Dit script laadt de data voor de analyse/modellen in, koppelt die, en doet benodigde bewerkingen om een datamodel te bouwen voor verder gebruik <br>

Input<br>
- Leefbaarometer, https://data.overheid.nl/dataset/leefbaarometer-2-0---meting-2016<br>
- CBS: buurten, wijken, gemeenten: Drie kleinere bestanden op https://www.cbs.nl/nl-nl/maatwerk/2017/38/buurt-wijk-en-gemeente-2017-voor-postcode-huisnummer<br>
- CBS: adressen koppelbestand: https://www.cbs.nl/nl-nl/maatwerk/2017/38/buurt-wijk-en-gemeente-2017-voor-postcode-huisnummer<br>
- CBS: kerncijfers wijken en buurten, zowel in CBSODATA als hier: https://www.cbs.nl/nl-nl/maatwerk/2017/31/kerncijfers-wijken-en-buurten-2017
- CBS: nabijheidsstatistieken over voorzieningen in buurten, zowel in CBSODATA als hier: https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/wijk-en-buurtstatistieken/nabijheidsstatistieken-per-wijk-buurt

- Calcasa: woningmarktprijsinformatie per wijk, Vesteda data<br>
- Calcasa: woningmarktprijsinformatie per gemeente, Vesteda data<br>
- Calcasa: woningvoorraad incl bouwjaren informatie per buurt, incl koop/huur splits, bestanden zijn gesplitst naar provincie, Vesteda data<br>
N.B. van Calcasa is alleen de wijk/buurt/gemeentenamen per heden beschikbaar. De bronbestanden bevatten de waarden van 2017Q4. 
Calcasa bestanden bevatten geen codes om te matchen, alleen namen. 


Geen 2017 versie van en daarom niet inlezen: 
- Vesteda interne data: MobilityLabel: bereikbaarheidstabel per PC5 en markpotentie per gemeente<br>
- ABF: data over midden en hoge inkomens per buurt (er is wel iets maar dat is alleen op gemeente niveau)<br>


Output, in 2017 subdir:
- bm = buurten main: Bestand met alle buurten van Nederland en informatie daarover; als input voor analyses van buurten gezamenlijk met wm en gm
- wm = wijken main
- gm = gemeenten main
- buurten: bm,wm,gm samengevoegd

### Declaraties

In [3]:
# importeren packages
# package odata installeren: pip install cbsodata (zie ook https://pypi.org/project/cbsodata/)
import cbsodata
import pandas as pd
import numpy as np

# setten source path: later moet alle brondata naar datalake
source_path ="P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Brondata\\2017\\"
# alle tussentijdse data voorlopig in dezelfde map op een schijf opslaan in
save_path = "P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\2017\\"

# jaar waarin gemeente op 1 januari een bevolking moet hebben om meegenomen te worden 
peiljaar = 2017

### Inlezen buurtcodes en buurtnamen
Bronbestand CBS van peiljaar inlezen (in dit geval 2017)

In [5]:
# inladen master bestand buurtcodes. Verschil tov 2019 data is dat hier een unicode escape nodig is anders foutmeldingen met speciale tekens.
buurten = pd.read_csv(source_path+'buurt'+str(peiljaar)+'.csv', sep=';', encoding='unicode_escape')
# hernoemen kolommen
buurten.columns=['buurtcode', 'buurtnaam']
# kolom maken met buurtcodeLetters ten behoeve van mergen met andere bronnen
buurten['buurtcodeLetters'] = 'BU' + buurten['buurtcode'].astype(str).str.zfill(8)

### Inlezen wijkcodes en wijknamen

In [6]:
wijken = pd.read_csv(source_path+'wijk'+str(peiljaar)+'.csv', sep=';', encoding='unicode_escape')
# hernoemen kolommen
wijken.columns=['wijkcode', 'wijknaam']

### Inlezen gemeentecodes en gemeentenamen

In [7]:
# inladen master bestand gemeenten
gemeenten = pd.read_csv(source_path+'gem'+str(peiljaar)+'.csv', sep=';', encoding='unicode_escape')
# hernoemen kolommen
gemeenten.columns=['gemeentecode', 'gemeentenaam']

### Wijkcodes, wijknamen, gemeentecodes, gemeentenamen toevoegen aan buurten<br>En gemeentecodes, gemeentenamen toevoegen aan wijken
Dit is gebaseerd op uitgangspunten: <br>
1. Een wijkcode is af te leiden uit de buurtcodes door de laatste twee cijfers van de buurtcode te verwijderen <br>
2. Een gemeentecode is af te leiden uit de wijkcode door de laatste twee cijfers van de wijkcode te verwijderen <br>
Dit moet altijd kloppen, ook na herindelingen.(voor 2019 situatie gecheckt met koppeltabel).

In [8]:
buurten['wijkcode'] = buurten['buurtcode'].astype(str).str[:-2].astype(np.int64)
buurten = pd.merge(buurten, wijken, how='left', on='wijkcode')

buurten['gemeentecode'] = buurten['wijkcode'].astype(str).str[:-2].astype(np.int64)
buurten = pd.merge(buurten, gemeenten, how='left', on='gemeentecode')

wijken['gemeentecode'] = wijken['wijkcode'].astype(str).str[:-2].astype(np.int64)
wijken = pd.merge(wijken, gemeenten, how='left', on='gemeentecode')

### Ophalen regionale kerncijfers van CBS ODATA

In [9]:
# dit tabelnummer 84583 is de versie van 2019; 83765 is de versie van 2017. 
kerncijfers = pd.DataFrame(cbsodata.get_data('83765NED'))

### Toevoegen regionale kerncijfers aan buurten. Alleen relevante kolommen. 
Het bronbestand regionale kerncijfers telt ruim 100 kolommen. Hiervan zijn er maar ca 5 nodig.<br>
Daarnaast alleen de rijen gebruiken die buurtinformatie bevatten, dus weggooien alle andere rijen.

In [10]:
# filteren alleen buurt: SoortRegio_2 = Buurt met 5 spaties
kerncijfersBuurt = kerncijfers.loc[kerncijfers['SoortRegio_2'] == 'Buurt     ']

# selecteren relevante kolommen
keepcols  = ['Codering_3', 'AantalInwoners_5', 'Bevolkingsdichtheid_33', 'Omgevingsadressendichtheid_105', 
             'HuishoudensTotaal_28', 'k_65JaarOfOuder_12']
kerncijfersBuurt = kerncijfersBuurt[keepcols]
# hernoemen kolomnamen
kerncijfersBuurt.columns=['buurtcodeLetters', 'inw', 'bevdh', 'oadh', 'hh', 'inw65' ]

### Introductie bm: buurten Main

In [11]:
# toevoegen kerncijfers aan masterbestand gematcht op buurtcodeLetters
# hier wordt bm geintroduceerd: buurten bestand MAIN. 
bm=pd.merge(buurten, kerncijfersBuurt, how='left', on='buurtcodeLetters')

### Ophalen leefbaarometer

In [12]:
# inlezen leefbaarheidsexcels
leefbaarheid=pd.read_excel(source_path+ 'Score_Buurt.xlsx', sheet_name='data')
dimensiescore=pd.read_excel(source_path+ 'dimensiescore_Buurt (stand).xlsx', sheet_name='data(2)')

### Toevoegen leefbaarometer op buurtniveau aan buurten
Kolommen selecteren en hernoemen<br>
Matchen<br>
Opmerkingen:
1. Er zijn buurten die niet meer bestaan in de CBS buurten maar nog wel in de leefbarometer Dat komt door herindelingen. Dit is waarschijnlijk de andere kant van het volgende punt<br>
2. Er bleken  buurten die wel bestaan in CBS buurten maar niet in leefbarometer. Itt 2019 data is dat hier voor 2017 niet op te lossen want er zijn geen historische buurtcodes<br>
3. daarnaast zijn er buurten andere grenzen hebben gekregen.

In [13]:
# selecteren relevante kolommen score
keepcols =['BU_CODE', 'KL14', 'KL16', 'VKL1416']
leefbaarheid=leefbaarheid[keepcols]

# hernoemen sleutelkolom tbv samenvoegen
leefbaarheid.columns = ['buurtcodeLetters', 'KL14', 'KL16', 'VKL1416']

# selecteren relevante kolommen dimensiescore
keepcols =['GBD', 'RLBRMTR14', 'RLBWON14', 'RLBBEV14','RLBVRZ14', 'RLBVEI14', 'RLBFYS14',
           'RLBRMTR16', 'RLBWON16', 'RLBBEV16', 'RLBVRZ16', 'RLBVEI16', 'RLBFYS16']

# hernoemen sleutelkolom tbv samenvoegen
dimensiescore=dimensiescore[keepcols]
dimensiescore.columns = ['buurtcodeLetters', 'RLBRMTR14', 'RLBWON14', 'RLBBEV14', 'RLBVRZ14', 'RLBVEI14', 'RLBFYS14',
                         'RLBRMTR16', 'RLBWON16', 'RLBBEV16', 'RLBVRZ16', 'RLBVEI16', 'RLBFYS16']

### Toevoegen leefbaarheidsinfo obv buurtode

In [14]:
# samenvoegen
bm=pd.merge(bm, leefbaarheid, how='left', on='buurtcodeLetters')
# toevoegen dimensiescore info
bm=pd.merge(bm, dimensiescore, how ='left', on= 'buurtcodeLetters')

### Ophalen voorzieningen informatie CBS
Deze tabel heet nabijheidsstatistieken en is nodig omdat diverse nabijheidsinformatie in de regionale kerncijfers leeg was. 
Versie 2017 heet 84334

In [15]:
nabij = pd.DataFrame(cbsodata.get_data('84334NED'))

### Toevoegen voorzieningen / nabijheids informatie aan buurten
er zijn hier ca 70 interessante indicatoren, hieronder een eerste selectie, wellicht tzt nog eens alle 70 bekijken en
beoordelen op samenhang!

In [16]:
# filteren alleen buurt: SoortRegio_2 = Buurt met 5 spaties
# LET OP hier wordt info op wijkniveau weggegooid, dat is jammer want er ontbreekt veel op buurtniveau!
nabij = nabij.loc[nabij['SoortRegio_2'] == 'Buurt     ']

# selecteren relevante kolommen. Totaal zijn er 70 waarvan veel overlap. Dit is een eerste selectie met interessante kandidaten.
# de kolomnamen 19 en 17 zijn gelukkig allemaal gelijk
keepcols  = ['Codering_3', 'AfstandTotGroteSupermarkt_24', 'Binnen3Km_30', 'AfstandTotHuisartsenpraktijk_5', 'Binnen3Km_38', 
             'Binnen3Km_46', 'Binnen1Km_53', 'AfstandTotZwembad_93', 'Binnen10Km_97', 'Binnen10Km_101', 'Binnen5Km_105']
nabij= nabij[keepcols]

# hernoemen kolomnamen
nabij.columns= ['buurtcodeLetters','AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3', 'AantalRest3', 
                     'AantalKinderopvang1', 'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5']

In [17]:
# hernoemen kolomnamen
nabij.columns= ['buurtcodeLetters','AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3', 'AantalRest3', 
                     'AantalKinderopvang1', 'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5']

In [18]:
# toevoegen aan buurtmain obv buurtcodeLetters
bm=pd.merge(bm, nabij, how='left', on='buurtcodeLetters')

### Ophalen Calcasa info wijk en gemeente
Huizenprijs ontwikkelingen zijn op wijkniveau veel completer beschikbaar dan op buurtniveau. Daarom op wijkniveau. 
Omdat er vergeleken gaat worden met gemeente niveau moet er ook info op gemeenteniveau worden ingelezen. 

Woningvoorraad en koop/huur split echter wel gebruiken op buurtniveau, omdat daar beschikbaarheid goed is.

In [19]:
# inlezen Calcasa xlsx wijken.
calcWijk = pd.read_excel(source_path + 'calcasa wijken 2017q4.xlsx', na_values='-')\
# verwijderen 1e regel die bevat de datum
calcWijk =  calcWijk.iloc[1:]
# hernoemen kolommen
calcWijk.columns = ['wijknaam', 'gemeentenaam', 'wox', 'woxdelta1jr', 'woxdelta5jr', 'woxww', 'woxm2']
# verwijderen de laatste regel die bevat Bron: Calcasa
calcWijk = calcWijk[~calcWijk.wijknaam.str.contains("Calcasa")]

# verwijderen rijen met alleen maar nullen want ca 100 wijken van de 1300 bevatten alleen maar nullen d.i. naN
# Dit zijn vnl bedrijventerreinen.
# door deze verwijdering vervalt ook het dubbel voorkomen van Kraggenburg-bedrijventerrein binnen dezelfde gemeente. 
calcWijk.dropna(axis=0, subset=['wox', 'woxdelta1jr', 'woxdelta5jr', 'woxww', 'woxm2'], how='all', inplace=True)

In [20]:
# inlezen Calcasa xlsx gemeenten. N.b. moet zelfde peildatum als wijken hebben 
calcGem = pd.read_excel(source_path + 'calcasa gemeenten 2017q4.xlsx', na_values='-')
# verwijderen 1e regel die bevat de datum
calcGem =  calcGem.iloc[1:]
calcGem.columns = ['gemeentenaam', 'woxGem', 'woxdelta1jrGem', 'woxdelta5jrGem', 'woxwwGem', 'woxm2Gem']
# verwijderen laatste regel die bevat bron: Calcasa
calcGem = calcGem[~calcGem.gemeentenaam.str.contains("Calcasa")]

### Afleiden nieuwe kolommen  die wijk en gemeente vergelijken Calcasa info

In [21]:
# toevoegen calcGem aan calcWijk
calcWijk=pd.merge(calcWijk, calcGem, how='left', on=['gemeentenaam'])
# afleiden kolom absoluut verschil 5jrs delta wijk en gem, uitkomst is een percentage
calcWijk['woxdelta5jrWGabs'] = calcWijk['woxdelta5jr'] - calcWijk['woxdelta5jrGem']
# afleiden kolom relatief verschil 5jrs delta wijk en gem
calcWijk['woxdelta5jrWGrel'] = calcWijk['woxdelta5jr'] / calcWijk['woxdelta5jrGem']
# afleiden kolom verschil m2 prijs wijk en gem
calcWijk['woxm2WG'] = calcWijk['woxm2'] / calcWijk['woxm2Gem']

### Toevoegen Calcasa info aan wijken

In [22]:
# introducie wm: wijken main
# matchen aan de hand van wijknaam en gemeentenaam. Code niet aanwezig dus dat kan niet. 
# En alleen matchen op wijknaam geeft dubbelen.  Daarom inclusief gemeentenaam. 
wm=pd.merge(wijken, calcWijk, how='left', on=['wijknaam', 'gemeentenaam'])

### Ophalen Calcasa op buurtniveau mbt woningvoorraad en koop/huur

In [23]:
# functie declareren die alle deelbestanden inleest en samenvoegt
def prov():
    # eerst de dfwv opbouwen met een eerste provincie
    provincie='NB'
    prov = pd.read_excel(source_path + 'calcasa wv bouwjaren 2017Q4 ' + provincie + '.xlsx', na_values='-')
    # verwijderen 1e regel die bevat de datum
    prov =  prov.iloc[1:]
    # hernoemen kolommen
    prov.columns = ['buurtnaam', 'gemeentenaam', 'wvtotaal', 'wvkoop', 'wvhuursoc', 'wvhuurov', 
                    'wv<1945', 'wv1945-1970', 'wv1970-1980', 'wv1980-1990', 'wv1990-2000', 'wv2000-2010', 'wv>=2010']
    # verwijderen laatste regel die bevat bron: Calcasa
    prov = prov[~prov.buurtnaam.str.contains("Calcasa")]
    #print(prov.head())
    #print('aantal rijen na verwerken van NB: ', len(prov))
    dfwv = prov
    
    # hier de dfwv steeds uitbreiden met een provincie
    for provincie in ['ZE', 'ZH', 'LI', 'OV', 'GE', 'DR', 'FL', 'FR', 'GR', 'NH', 'UT']:
        prov = pd.read_excel(source_path + 'calcasa wv bouwjaren 2017Q4 ' + provincie + '.xlsx', na_values='-')
        # verwijderen 1e regel die bevat de datum
        prov =  prov.iloc[1:]
        # hernoemen kolommen
        prov.columns = ['buurtnaam', 'gemeentenaam', 'wvtotaal', 'wvkoop', 'wvhuursoc', 'wvhuurov', 
                        'wv<1945', 'wv1945-1970', 'wv1970-1980', 'wv1980-1990', 'wv1990-2000', 'wv2000-2010', 'wv>=2010']
        # verwijderen laatste regel die bevat bron: Calcasa
        prov = prov[~prov.buurtnaam.str.contains("Calcasa")]
        # toevoegen aan main
        dfwv = dfwv.append(prov, ignore_index=True)
        # LAB versie statement
        #print('aantal rijen na verwerken van', provincie, ': ', len(dfwv))
    return dfwv

In [24]:
#aanroepen van de functie
dfwv=prov()

In [25]:
# merge sleutel maken. Hier wordt gematcht op buurtnaam. 
dfwv['sleutel'] = dfwv['gemeentenaam']+dfwv['buurtnaam']
bm['sleutel'] = bm['gemeentenaam']+bm['buurtnaam']

In [26]:
# verwijderen rijen die dubbel voorkomen
# verwijderen kolommen

eruit = dfwv[dfwv.duplicated(['sleutel'])]
# de dubbele sleutels in wv moeten worden opgelost. bv. Sterrenbuurt in Lansingerland: betreft 2 andere plaatsen. Dimensie wijk 
# erbij was handig geweest! Voor nu makkelijkste oplossing eruit gooien. 
dfwv =  dfwv[~dfwv.sleutel.isin(eruit['sleutel'])]

# verwijderen gemeentenaam en buurtnaam uit dfwv anders worden deze kolommen gedupliceerd
dfwv.drop(columns=['gemeentenaam', 'buurtnaam'], inplace=True)

In [27]:
# kolommen toeovegen voor de procentuele aandelen van elke bouwperiode en koop/huur verdeling
dfwv.fillna(0, inplace = True) 
dfwv['aand<1945'] = dfwv['wv<1945'] / dfwv['wvtotaal']
dfwv['aand1945-1970'] = dfwv['wv1945-1970'] / dfwv['wvtotaal']
dfwv['aand1970-1980'] = dfwv['wv1970-1980'] / dfwv['wvtotaal']
dfwv['aand1980-1990'] = dfwv['wv1980-1990'] / dfwv['wvtotaal']
dfwv['aand1990-2000'] = dfwv['wv1990-2000'] / dfwv['wvtotaal']
dfwv['aand2000-2010'] = dfwv['wv2000-2010'] / dfwv['wvtotaal']
dfwv['aand>=2010'] = dfwv['wv>=2010'] / dfwv['wvtotaal']
dfwv['aandkoop'] = dfwv['wvkoop'] / dfwv['wvtotaal']
dfwv['aandhuursoc'] = dfwv['wvhuursoc'] / dfwv['wvtotaal']
dfwv['aandhuurov'] = dfwv['wvhuurov'] / dfwv['wvtotaal']

In [28]:
# toevoegen wv info aan bm
bm = pd.merge(bm, dfwv, how='left', on=['sleutel'])

### gemeenten
Geen bewerkingen want geen MP in de 2017 versie

In [27]:
gm = gemeenten

### Opslaan resultaten als csv

In [28]:
# drie basisbestanden
bm.to_csv(save_path+'bm.csv', encoding='utf-16', index=False)
wm.to_csv(save_path+'wm.csv', encoding='utf-16', index=False)
gm.to_csv(save_path+'gm.csv', encoding='utf-16', index=False)

In [29]:
# samengestelde csv. om dubbele kolommen te voorkomen eerst droppen bepaalde kolommen
wm.drop(columns=['wijknaam', 'gemeentecode', 'gemeentenaam'], inplace=True)
gm.drop(columns=['gemeentenaam'], inplace=True)

buurten=pd.merge(bm, wm, how='left', on ='wijkcode')
buurten=pd.merge(buurten, gm, how='left', on ='gemeentecode')

buurten.to_csv(save_path+'buurten.csv', encoding='utf-16', index=False)